In [131]:
import json
import pandas as pd
from collections import Counter
from statistics import mean
import numpy as np

In [2]:
df = pd.read_json("imdb_dataset_parsed.json")

In [3]:
df.head(3)

,title,rating,year,users_rating,votes,metascore,actors,genre,directors,runtime
0,12 Strong,R,2018.0,6.6,"42,919",54.0,"[Chris Hemsworth, Michael Shannon, Michael PeÃ...","[Action, Drama, History, War]",[Nicolai Fuglsig],130 min
1,The Silence of the Lambs,R,1991.0,8.6,"1,091,846",85.0,"[Jodie Foster, Lawrence A. Bonney, Kasi Lemmon...","[Crime, Drama, Thriller]",[Jonathan Demme],118 min
2,Blade Runner,R,1982.0,8.2,"609,925",89.0,"[Harrison Ford, Rutger Hauer, Sean Young, Edwa...","[Sci-Fi, Thriller]",[Ridley Scott],117 min


In [4]:
df = df.dropna()

In [89]:
directors_list = []
for directors in df["directors"]:
    for director in directors:
        directors_list.append(director)

In [90]:
directors_set = set(directors_list)
directors_list = list(directors_set)

In [91]:
my_dict = dict((el,[]) for el in directors_list)
for index,row in df.iterrows():
    for director in row["directors"]:
        my_dict[director].append(row["actors"])

In [92]:
final_dict = {}
for k,v in my_dict.items():
    final_dict[k] = [item for sublist in v for item in sublist]

In [93]:
for k in final_dict.keys():
    final_dict[k] = Counter(final_dict[k]).most_common(1)

In [95]:
for index,row in df.iterrows():
    for director in row["directors"]:
        for actor in row["actors"]:
            if(actor in final_dict[director][0][0]):
                final_dict[director].append(row["metascore"])

In [101]:
new_dict = {}
for k,v in final_dict.items():
    try:
        if final_dict[k][0][1] > 4:
            new_dict[k] = [final_dict[k][0][0],final_dict[k][0][1], final_dict[k][1:]]
    except:
        continue

In [156]:
df_couples = pd.DataFrame.from_dict(new_dict, orient='index')
df_couples.reset_index(level=0, inplace=True)
df_couples.columns = ["director", "actor", "number_of_films", "metascore_mean"]

In [158]:
mean_scores = []
for index,row in df_couples.iterrows():
    mean_scores.append(mean(row["metascore_mean"]))

In [159]:
df_couples["metascore_mean"] = mean_scores

In [163]:
df_couples

,director,actor,number_of_films,metascore_mean
0,Lawrence Kasdan,Kevin Kline,5,56.800000
1,James Wan,Patrick Wilson,5,56.000000
2,Alex Ross Perry,Kate Lyn Sheil,5,61.800000
3,Darren Aronofsky,Mark Margolis,6,69.833333
4,Tony Scott,Denzel Washington,5,59.200000
5,John R. Cherry III,Jim Varney,5,34.800000
6,Woody Allen,Woody Allen,18,69.055556
7,Steven Spielberg,Tom Hanks,5,76.800000
8,Robert Rodriguez,Danny Trejo,7,56.714286
9,Danny DeVito,Danny DeVito,5,59.000000


In [162]:
#df_couples.to_json("couples.json", orient='records')